In [1]:
import pandas as pd
import numpy as np
import os
import glob
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


In [2]:
bednames = ['chrom','start','end','name','score','strand']
brian = pd.read_table('/home/bay001/projects/codebase/annotator/examples/create_region_bedfiles/outputs/hg19_v19_cds.bed', 
                      names=['chrom','start','end','name','score','strand'])
print(brian.shape)
brian['start'] = brian['start'] + 1
gabe = pd.read_table(
    '/home/bay001/projects/codebase/clip_analysis_legacy/clip_analysis_legacy/data/regions/hg19_v19_cds.bed',
    names=bednames
)
print(gabe.shape)
brian.head()

(213027, 6)
(213027, 6)


,chrom,start,end,name,score,strand
0,chrY,21867884,21868231,ENSG00000012817.11,0,-
1,chrY,21868327,21868526,ENSG00000012817.11,0,-
2,chrY,21868680,21868749,ENSG00000012817.11,0,-
3,chrY,21869033,21869632,ENSG00000012817.11,0,-
4,chrY,21869820,21869957,ENSG00000012817.11,0,-


In [3]:
gabe.head()

,chrom,start,end,name,score,strand
0,chr1,69091,70005,ENSG00000186092.4,0,+
1,chr1,138533,139309,ENSG00000237683.5,0,-
2,chr1,367659,368594,ENSG00000235249.1,0,+
3,chr1,621099,622034,ENSG00000185097.2,0,-
4,chr1,738532,738618,ENSG00000269831.1,0,-


In [4]:
df = pd.merge(brian, gabe, how='outer', left_on=['chrom','start','end','strand'], right_on=['chrom','start','end','strand'])
df.head()

,chrom,start,end,name_x,score_x,strand,name_y,score_y
0,chrY,21867884,21868231,ENSG00000012817.11,0,-,ENSG00000012817.11,0
1,chrY,21868327,21868526,ENSG00000012817.11,0,-,ENSG00000012817.11,0
2,chrY,21868680,21868749,ENSG00000012817.11,0,-,ENSG00000012817.11,0
3,chrY,21869033,21869632,ENSG00000012817.11,0,-,ENSG00000012817.11,0
4,chrY,21869820,21869957,ENSG00000012817.11,0,-,ENSG00000012817.11,0


In [5]:
# CDS identical
df[df.isnull().any(axis=1)]

,chrom,start,end,name_x,score_x,strand,name_y,score_y


In [60]:
bednames = ['chrom','start','end','name','score','strand']
brian = pd.read_table('/home/bay001/projects/codebase/annotator/examples/create_region_bedfiles/outputs/hg19_v19_proxintrons.bed', 
                      names=['chrom','start','end','name','score','strand'])
print(brian.shape)
# brian['start'] = brian['start'] + 1
# brian['end'] = brian['end'] + 1
gabe = pd.read_table(
    '/home/bay001/projects/codebase/clip_analysis_legacy/clip_analysis_legacy/data/regions/hg19_v19_proxintron500.bed',
    names=bednames
)
print(gabe.shape)
brian.tail()

(402055, 6)
(412309, 6)


,chrom,start,end,name,score,strand
402050,chr8,146225924,146226424,proxintron500,.,-
402051,chr8,146227702,146228202,proxintron500,.,-
402052,chr8,146277883,146278132,proxintron500,.,+
402053,chr8,146278283,146278682,proxintron500,.,+
402054,chr8,146278829,146279403,proxintron500,.,+


In [61]:
gabe[gabe['name']=='ENSGR0000228572.2']

,chrom,start,end,name,score,strand
409640,chrY,120514,121014,ENSGR0000228572.2,0,+
409641,chrY,121103,121603,ENSGR0000228572.2,0,+


In [62]:
df = pd.merge(brian, gabe, how='outer', left_on=['chrom','start','end','strand'], right_on=['chrom','start','end','strand'])
df.head()

,chrom,start,end,name_x,score_x,strand,name_y,score_y
0,chrY,120513,121013,proxintron500,.,+,NaN,NaN
1,chrY,121103,121603,proxintron500,.,+,ENSGR0000228572.2,0.0
2,chrY,143061,143561,proxintron500,.,+,NaN,NaN
3,chrY,147648,148148,proxintron500,.,+,ENSGR0000182378.8,0.0
4,chrY,148351,148851,proxintron500,.,+,NaN,NaN


In [55]:
df[df.isnull().any(axis=1)].shape

(500555, 8)

In [59]:
gabe[gabe['end']==121603]

,chrom,start,end,name,score,strand
409641,chrY,121103,121603,ENSGR0000228572.2,0,+


In [10]:
import gffutils
import os
import pybedtools
import pandas as pd


In [11]:
dx = pd.read_table(
    '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf',
    comment='#',
    names=range(9)
)
dx.head()

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENSG00000223972.4""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""pseudogene""; transcript_status ""KNOWN""; transcript_name ""DDX11L1""; level 2; havana_gene ""OTTHUMG00000000961.2"";"
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 1; exon_id ""ENSE00002234944.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 2; exon_id ""ENSE00003582793.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 3; exon_id ""ENSE00002312635.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"


In [12]:
dx['gene'] = dx[8].str.extract("gene_id \"([\w\d\.]+)\"")
dx.head()

/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,0,1,2,3,4,5,6,7,8,gene
0,chr1,HAVANA,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENSG00000223972.4""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""pseudogene""; transcript_status ""KNOWN""; transcript_name ""DDX11L1""; level 2; havana_gene ""OTTHUMG00000000961.2"";",ENSG00000223972.4
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";",ENSG00000223972.4
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 1; exon_id ""ENSE00002234944.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";",ENSG00000223972.4
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 2; exon_id ""ENSE00003582793.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";",ENSG00000223972.4
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 3; exon_id ""ENSE00002312635.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";",ENSG00000223972.4


In [13]:
dx[dx['gene']=='ENSGR0000228572.2']

,0,1,2,3,4,5,6,7,8,gene
2610229,chrY,HAVANA,gene,120410,121758,.,+,.,"gene_id ""ENSGR0000228572.2""; transcript_id ""ENSGR0000228572.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""LL0YNC03-29C1.1""; transcript_type ""pseudogene""; transcript_status ""KNOWN""; transcript_name ""LL0YNC03-29C1.1""; level 2; havana_gene ""OTTHUMG00000046372.2""; tag ""PAR"";",ENSGR0000228572.2
2610230,chrY,HAVANA,transcript,120410,121758,.,+,.,"gene_id ""ENSGR0000228572.2""; transcript_id ""ENSTR0000431238.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""LL0YNC03-29C1.1""; transcript_type ""unprocessed_pseudogene""; transcript_status ""KNOWN""; transcript_name ""LL0YNC03-29C1.1-001""; level 2; ont ""PGO:0000005""; havana_gene ""OTTHUMG00000046372.2""; havana_transcript ""OTTHUMT00000107057.2""; tag ""PAR"";",ENSGR0000228572.2
2610231,chrY,HAVANA,exon,120410,120513,.,+,.,"gene_id ""ENSGR0000228572.2""; transcript_id ""ENSTR0000431238.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""LL0YNC03-29C1.1""; transcript_type ""unprocessed_pseudogene""; transcript_status ""KNOWN""; transcript_name ""LL0YNC03-29C1.1-001""; exon_number 1; exon_id ""ENSE00001702291.1""; level 2; ont ""PGO:0000005""; havana_gene ""OTTHUMG00000046372.2""; havana_transcript ""OTTHUMT00000107057.2""; tag ""PAR"";",ENSGR0000228572.2
2610232,chrY,HAVANA,exon,121604,121758,.,+,.,"gene_id ""ENSGR0000228572.2""; transcript_id ""ENSTR0000431238.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""LL0YNC03-29C1.1""; transcript_type ""unprocessed_pseudogene""; transcript_status ""KNOWN""; transcript_name ""LL0YNC03-29C1.1-001""; exon_number 2; exon_id ""ENSE00001655436.1""; level 2; ont ""PGO:0000005""; havana_gene ""OTTHUMG00000046372.2""; havana_transcript ""OTTHUMT00000107057.2""; tag ""PAR"";",ENSGR0000228572.2


In [14]:
bedtool = bedtool.sort()

NameError: name 'bedtool' is not defined

In [ ]:
df = bedtool.merge(s=True, c=(2,3,4), o='distinct').to_dataframe()

In [ ]:
df[df['start']==148579637]

In [ ]:
dx = pd.concat([pd.Series(zip(row['score'], row['strand'].split(','), row['thickStart'].split(','))) for _, row in df.iterrows()]).reset_index()

In [ ]:
dx[dx[0]=='148579637']

In [ ]:
from itertools import izip_longest
a=['a1']
b=['b1','b2','b3']
c=['c1','c2']

for x in izip_longest(a, b, c, ):
    print(x)


In [ ]:
bedtools groupby -g 1,4,5,6 -c 2,3 -o min,max | awk -v OFS="\t" '{print $1,$5,$6,$2,$3,$4}'

In [ ]:
df = bedtool.to_dataframe()
from tqdm import tnrange, tqdm_notebook
progress = tnrange(len(set(df['name'])))
for name in set(df['name']):
    dx = df[df['name']==name]
        bt = pybedtools.BedTool.from_dataframe(dx)
    bt = bt.merge(s=True).to_dataframe()
    progress.update(1)

In [67]:
bednames = ['chrom','start','end','name','score','strand']
brian = pd.read_table('/home/bay001/projects/codebase/annotator/examples/create_region_bedfiles/outputs/hg19_v19_five_prime_utrs.bed', 
                      names=['chrom','start','end','name','score','strand'])
print(brian.shape)
brian['start'] = brian['start'] + 1
# brian['end'] = brian['end'] + 1
gabe = pd.read_table(
    '/home/bay001/projects/codebase/clip_analysis_legacy/clip_analysis_legacy/data/regions/hg19_v19_five_prime_utrs.bed',
    names=bednames
)
print(gabe.shape)
brian[brian['chrom']=='chr1'].sort_values('start').head()

(59682, 6)
(59534, 6)


,chrom,start,end,name,score,strand
55071,chr1,139310,139379,ENSG00000237683.5,0,-
55058,chr1,367640,367658,ENSG00000235249.1,0,+
54217,chr1,622035,622053,ENSG00000185097.2,0,-
54402,chr1,860260,860328,ENSG00000187634.6,0,+
54403,chr1,860530,860569,ENSG00000187634.6,0,+


In [68]:
gabe[gabe['chrom']=='chr1'].head()

,chrom,start,end,name,score,strand
0,chr1,139310,139379,ENSG00000237683.5,0,-
1,chr1,367640,367658,ENSG00000235249.1,0,+
2,chr1,622035,622053,ENSG00000185097.2,0,-
3,chr1,860260,860328,ENSG00000187634.6,0,+
4,chr1,860530,860569,ENSG00000187634.6,0,+


In [69]:
df = pd.merge(brian, gabe, how='outer', left_on=['chrom','start','end','strand'], right_on=['chrom','start','end','strand'])
df.head()

,chrom,start,end,name_x,score_x,strand,name_y,score_y
0,chrY,21906421,21906439,ENSG00000012817.11,0,-,ENSG00000012817.11,0.0
1,chrY,21906557,21906825,ENSG00000012817.11,0,-,ENSG00000012817.11,0.0
2,chrY,15016019,15016325,ENSG00000067048.12,0,+,ENSG00000067048.12,0.0
3,chrY,15016742,15016847,ENSG00000067048.12,0,+,ENSG00000067048.12,0.0
4,chrY,15017649,15017690,ENSG00000067048.12,0,+,ENSG00000067048.12,0.0


In [70]:
df[df.isnull().any(axis=1)]

,chrom,start,end,name_x,score_x,strand,name_y,score_y
2263,chrX,101715518,101715616,ENSG00000215029.4,0,-,NaN,NaN
10697,chr11,6518815,6518897,ENSG00000265264.1,0,+,NaN,NaN
12077,chr10,102276631,102276717,ENSG00000166136.11,0,-,NaN,NaN
12078,chr10,102276631,102276717,ENSG00000255339.2,0,-,NaN,NaN
16004,chr17,15638244,15638384,ENSG00000187607.11,0,+,NaN,NaN
16005,chr17,15638244,15638384,ENSG00000255104.3,0,+,NaN,NaN
16430,chr17,7297423,7297586,ENSG00000262481.1,0,-,NaN,NaN
19140,chr16,19184082,19184099,ENSG00000261427.2,0,+,NaN,NaN
19141,chr16,19184752,19184900,ENSG00000261427.2,0,+,NaN,NaN
19146,chr16,71487146,71487253,ENSG00000261611.3,0,-,NaN,NaN


In [75]:
dx[dx['gene']=='ENSG00000167807.11'].sort_values(3)

,0,1,2,3,4,5,6,7,8,gene
2256183,chr19,HAVANA,gene,10416103,10426685,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENSG00000167807.11""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""protein_coding""; transcript_status ""NOVEL""; transcript_name ""CTD-2369P2.10""; level 2; havana_gene ""OTTHUMG00000180407.2"";",ENSG00000167807.11
2256184,chr19,HAVANA,transcript,10416103,10426685,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256210,chr19,HAVANA,UTR,10416103,10416159,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256202,chr19,HAVANA,exon,10416103,10416159,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; exon_number 11; exon_id ""ENSE00003689157.1""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256209,chr19,HAVANA,UTR,10416522,10416628,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256201,chr19,HAVANA,exon,10416522,10416628,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; exon_number 10; exon_id ""ENSE00003554933.1""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256208,chr19,HAVANA,UTR,10418861,10419067,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT00000451222.1"";",ENSG00000167807.11
2256200,chr19,HAVANA,exon,10418861,10419067,.,-,.,"gene_id ""ENSG00000167807.11""; transcript_id ""ENST00000452032.2""; gene_type ""protein_coding""; gene_status ""NOVEL""; gene_name ""CTD-2369P2.10""; transcript_type ""nonsense_mediated_decay""; transcript_status ""KNOWN""; transcript_name ""CTD-2369P2.10-001""; exon_number 9; exon_id ""ENSE00003506672.1""; level 2; tag ""basic""; tag ""appris_principal""; tag ""readthrough_transcript""; havana_gene ""OTTHUMG00000180407.2""; havana_transcript ""OTTHUMT0000045